<a href="https://colab.research.google.com/github/raphaobdd/analise_da_bolsa/blob/main/an_lise_da_bolsa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [ ]:
def analisar_acao(ticker, periodo_anos=5):
    # Adiciona .SA se não existir (para ações brasileiras)
    if '.SA' not in ticker.upper():
        ticker = ticker.upper() + '.SA'

    try:
        # Baixa dados históricos
        acao = yf.Ticker(ticker)
        hoje = datetime.now()
        inicio = hoje - timedelta(days=periodo_anos*365)

        # Dados históricos para análise técnica
        hist = acao.history(start=inicio, end=hoje)

        # Dados fundamentalistas
        info = acao.info

        # === Análise Fundamentalista ===
        analise = {}

        # P/L (Price/Earnings)
        try:
            analise['P/L'] = info.get('trailingPE', None)
            analise['P/L_viavel'] = 0 <= analise['P/L'] <= 20 if analise['P/L'] else None
        except:
            analise['P/L'] = None
            analise['P/L_viavel'] = None

        # P/VP (Price/Book)
        try:
            analise['P/VP'] = info.get('priceToBook', None)
            analise['P/VP_viavel'] = 0 <= analise['P/VP'] <= 2 if analise['P/VP'] else None
        except:
            analise['P/VP'] = None
            analise['P/VP_viavel'] = None

        # ROE (Return on Equity)
        try:
          analise['ROE'] = info.get('returnOnEquity', None)
          if analise['ROE']:
              analise['ROE'] = analise['ROE'] * 100
          analise['ROE_viavel'] = analise['ROE'] >= 10 if analise['ROE'] else None
        except:
          analise['ROE'] = None
          analise['ROE_viavel'] = None

        # Dividend Yield
        try:
            analise['DY'] = info.get('dividendYield', None)
            if analise['DY']:
                analise['DY'] = analise['DY'] * 100
            analise['DY_viavel'] = analise['DY'] >= 5 if analise['DY'] else None
        except:
            analise['DY'] = None
            analise['DY_viavel'] = None

        # === Análise Técnica ===

        # Média Móvel 200 dias
        hist['MM200'] = hist['Close'].rolling(window=200).mean()
        preco_atual = hist['Close'][-1]
        mm200_atual = hist['MM200'][-1]
        analise['Preço/MM200'] = preco_atual/mm200_atual
        analise['Preço_Acima_MM200'] = preco_atual > mm200_atual

        # Volatilidade (Desvio Padrão dos Retornos)
        retornos = hist['Close'].pct_change()
        analise['Volatilidade'] = retornos.std() * np.sqrt(252) * 100  # Anualizada
        analise['Volatilidade_viavel'] = analise['Volatilidade'] <= 40

        # === Pontuação Final ===
        pontos_possiveis = 0
        pontos_obtidos = 0

        for indicador in ['P/L_viavel', 'P/VP_viavel', 'ROE_viavel', 'DY_viavel', 'Preço_Acima_MM200', 'Volatilidade_viavel']:
            if analise[indicador] is not None:
                pontos_possiveis += 1
                if analise[indicador]:
                    pontos_obtidos += 1

        if pontos_possiveis > 0:
            analise['Pontuação'] = (pontos_obtidos / pontos_possiveis) * 100
        else:
            analise['Pontuação'] = None

        # Recomendação
        if analise['Pontuação'] is not None:
            if analise['Pontuação'] >= 80:
                analise['Recomendação'] = 'FORTE COMPRA'
            elif analise['Pontuação'] >= 60:
                analise['Recomendação'] = 'COMPRA'
            elif analise['Pontuação'] >= 40:
                analise['Recomendação'] = 'NEUTRO'
            elif analise['Pontuação'] >= 20:
                analise['Recomendação'] = 'VENDA'
            else:
                analise['Recomendação'] = 'FORTE VENDA'
        else:
            analise['Recomendação'] = 'DADOS INSUFICIENTES'

        return analise

    except Exception as e:
        return {
            'erro': f'Erro ao analisar ação: {str(e)}',
            'Recomendação': 'ERRO NA ANÁLISE'
        }


In [ ]:
def imprimir_analise(ticker):
    resultado = analisar_acao(ticker)

    print(f"\nAnálise da ação {ticker.upper()}")
    print("-" * 50)

    if 'erro' in resultado:
        print(f"ERRO: {resultado['erro']}")
        return

    # Indicadores Fundamentalistas
    print("\nIndicadores Fundamentalistas:")
    if resultado['P/L'] is not None:
        print(f"P/L: {resultado['P/L']:.2f} ({'✓' if resultado['P/L_viavel'] else '✗'})")
    if resultado['P/VP'] is not None:
        print(f"P/VP: {resultado['P/VP']:.2f} ({'✓' if resultado['P/VP_viavel'] else '✗'})")
    if resultado['ROE'] is not None:
        print(f"ROE: {resultado['ROE']:.2f}% ({'✓' if resultado['ROE_viavel'] else '✗'})")
    if resultado['DY'] is not None:
        print(f"Dividend Yield: {resultado['DY']:.2f}% ({'✓' if resultado['DY_viavel'] else '✗'})")

    # Indicadores Técnicos
    print("\nIndicadores Técnicos:")
    print(f"Preço/MM200: {resultado['Preço/MM200']:.2f} ({'✓' if resultado['Preço_Acima_MM200'] else '✗'})")
    print(f"Volatilidade Anual: {resultado['Volatilidade']:.2f}% ({'✓' if resultado['Volatilidade_viavel'] else '✗'})")

    # Resultado Final
    print("\nResultado Final:")
    if resultado['Pontuação'] is not None:
        print(f"Pontuação: {resultado['Pontuação']:.2f}%")
    print(f"Recomendação: {resultado['Recomendação']}")

In [ ]:
imprimir_analise('MGLU3')


Análise da ação MGLU3
--------------------------------------------------

Indicadores Fundamentalistas:
P/L: 11.40 (✓)
P/VP: 0.47 (✓)
ROE: -4.50% (✗)

Indicadores Técnicos:
Preço/MM200: 0.66 (✗)
Volatilidade Anual: 70.93% (✗)

Resultado Final:
Pontuação: 40.00%
Recomendação: NEUTRO


<ipython-input-18-f808c8400809>:61: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  preco_atual = hist['Close'][-1]
<ipython-input-18-f808c8400809>:62: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  mm200_atual = hist['MM200'][-1]
